In [ ]:
import os
from autogen import ConversableAgent, register_function, initiate_chats
from dotenv import load_dotenv
from typing import Annotated
import httpx

In [ ]:
OPENAI_API_KEY = "<insert-openai-api-key>"
OPENWEATHER_API_KEY = "<insert-openweather-api-key>"

In [ ]:
llm_config = {
        "config_list": [{"model": "gpt-4o-mini", "api_key": OPENAI_API_KEY }],
        "cache_seed": None,
    }

weather_assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI weather assistant. "
    "You provide relevant recommendations to the user based on available weather data. "
    "Return 'TERMINATE' after you've answered the user's question.",
    llm_config=llm_config,
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [ ]:
weather_api_proxy = ConversableAgent(
    name="Weather API Proxy",
    llm_config=False,
    default_auto_reply="Make an API request to get the latest weather as needed",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=1,
)

In [ ]:
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None
    or "terminate" in msg.get("content").lower(),
    human_input_mode="ALWAYS",
)

In [ ]:
def get_current_weather(
    lat: Annotated[float, "Latitude of the weather location"],
    lon: Annotated[float, "Longitude of the weather location"],
) -> dict:

    api_key = OPENWEATHER_API_KEY
    base_url = "https://api.openweathermap.org/data/3.0/onecall"
    arguments = f"?lat={lat}&lon={lon}&appid={api_key}&units=metric"

    endpoint = f"{base_url}{arguments}"
    response = httpx.get(endpoint)
    
    return response.json()

register_function(
    get_current_weather,
    caller=weather_assistant,
    executor=weather_api_proxy,
    description="A tool for obtaining weather information. Units are given in metric.",
)


In [ ]:
weather_assistant.register_nested_chats(
    trigger=user_proxy,
    chat_queue = [
        {
            "sender": weather_api_proxy,
            "recipient": weather_assistant,
            "summary_method": "reflection_with_llm",
        }
    ]
)

In [ ]:
chats = [
    {
        "sender": weather_assistant,
        "recipient": user_proxy,
        "message":
            "Hello, I'm here to provide recommendations based on the current weather."
            "How may I help you?",
        "summary_method": None,
        "clear_history": True,
        "max_turns": 2,
    }
]

In [ ]:
initiate_chats(chats)